# 0/1 Knapsack
You are a thief planning to rob a store. However, you can only carry a knapsack with a maximum capacity of cap units. Each item (i) in the store has a weight (weights[i]) and a value (values[i]).

Find the maximum total value of items you can carry in your knapsack.

**Example:**
```python
Input: cap = 7, weights = [5, 3, 4, 1], values = [70, 50, 40, 10]
Output: 90
```

Explanation: The most valuable combination of items that can fit in the knapsack together are items 1 and 2 . These items have a combined value of 50 + 40 = 90 and a total weight of 3 + 4 = 7 , which fits within the knapsack's capacity.

## Intuition
For each item, we have two choices: include it in the knapsack or exclude it. This binary decision is why this classic problem is called "0/1 Knapsack."

The brute force approach involves making this decision for every item. Since two choices can be made for each item, this results in:

$$ 2 \times 2 \times 2 \times ... \times 2 = 2^n $$

possible combinations of choices. As we can see, generating all possible combinations is inefficient.

A greedy solution that involves picking the most valuable items first isn't a good choice either, as it doesn't always lead to the optimal outcome.

So we can approach this problem from a different angle. What's the most value we can attain with a knapsack of capacity 7, if we include the first item? What about if we exclude this item? Let's define the function:

$$ knapsack(i, cap) $$

to represent the maximum value achievable with items starting from index \( i \) and a knapsack capacity of \( cap \).

We explore the implications of including or excluding this item separately.

---

### Including item \( i \)
Picking the first item gives a value of 70. This item weighs 5, so our knapsack now has a remaining capacity of:

$$ 7 - 5 = 2 $$

With an updated capacity of 2, what's the optimal combination possible with the remaining items in our selection?

This question leads us to realize that if we determine the maximum value that can be obtained from the remaining items (starting from index 1) with a knapsack capacity of 2, we can find the solution:

$$ 70 + knapsack(i = 1, cap = 7 - 5 = 2) $$

We've identified that this case can be solved by solving a subproblem, which means we're dealing with a problem that has an optimal substructure.

Therefore, we can generalize a recurrence relation for this case. Below, \( c \) denotes the remaining knapsack capacity we want to solve for (we give it a different name because it can be different from the initial value of \( cap \)).

If we include item \( i \), the most value we can get is:

$$ values[i] + knapsack(i + 1, c - weights[i]) $$

---

### Excluding item \( i \)
Now, let's say we exclude item 0. This means our knapsack will maintain a capacity of 7. Here, the most value we can get is just from the maximum value from the rest of the items, with a knapsack of capacity 7.

If we exclude item \( i \), the most value we can get is:

$$ knapsack(i + 1, c) $$

---

Now that we've established the recurrence relations for both cases (including and excluding item \( i \)), we can combine them: the maximum value we can get from any selection of items is the larger value obtained from these two cases:

$$ knapsack(i, c) = \max(\text{include item } i, \text{ exclude item } i) $$

$$ = \max(values[i] + knapsack(i + 1, c - weights[i]), knapsack(i + 1, c)) $$

One case we haven't covered yet is the possibility that the item does not fit in the knapsack. In this case, we have no choice but to exclude the item, resulting in:

$$ knapsack(i + 1, c) $$

---

### Dynamic Programming
Given that this problem has an optimal substructure, we can translate our recurrence relations directly into DP formulas:

If the item's weight is less than or equal to the remaining capacity:

$$ dp[i][c] = \max(values[i] + dp[i + 1][c - weights[i]], dp[i + 1][c]) $$

Otherwise:

$$ dp[i][c] = dp[i + 1][c] $$

For clarity, there are two dimensions in our DP table:
- One for the current item, \( i \), represented by the rows of the DP table.
- One for the current knapsack capacity, \( c \), represented by the columns of the DP table.

---

### Base Cases
The simplest version of this problem is when there are no items in our selection, meaning the maximum value we can attain is 0. But which cells of the DP table represent this base case?

We know that when:

$$ i = n - 1 $$

only one item from the selection is being considered (where \( n \) denotes the total number of items).

This implies that when:

$$ i = n $$

no items are being considered. Therefore, we can populate the DP table using:

$$ dp[n][c] = 0 \quad \text{for all } c $$

The reason \( i = 0 \) isn't the base case is because \( i = 0 \) encapsulates all items starting from index 0.

Another subproblem we know the answer to is:

$$ c = 0 $$

since no items can fit in a knapsack of capacity 0. For this, we can populate the DP table using:

$$ dp[i][0] = 0 \quad \text{for all } i $$

So the first column and last row are set to 0 for the base cases.

---

### Populating the DP Table
We populate the DP table starting from the smallest subproblems (excluding base cases). Specifically, this means starting from:

$$ i = n - 1 $$

where only the last item is considered, and ending at:

$$ i = 0 $$

where we consider all items. For each of these rows, we iterate through each possible knapsack capacity from:

$$ c = 1 \text{ to } c = cap $$

In [1]:
from typing import List

def knapsack(k: int, weights: List[int], values: List[int]) -> int:
    n = len(values)
    dp = [[0 for x in range(k + 1)] for x in range(n + 1)]

    for i in range(n - 1, -1, -1):
        curr_row = [0] * (k + 1)

        for c in range(1, k + 1):

            if weights[i] <= c:
                dp[i][c] = max(values[i] + dp[i + 1][c - weights[i]], dp[i + 1][c])
            else:
                dp[i][c] = dp[i + 1][c]
            
    return dp[0][k]

### Complexity Analysis
The time complexity is O(n * cap) because each cell of the DP table is populated once.

The space complexity is O(n * cap) because we maintain a DP table that stores (n + 1)(cap + 1) elements.

---

## Optimization
We can optimize the solution by recognizing that, for each cell in the DP table, we only need access cells from the row below it.

Therefore, we only need to maintain two rows:
- curr_row: the current row being populated.
- prev_row: the row below the current row.

This effectively reduces the space complexity to O(cap).

In [2]:
from typing import List

def knapsack(k: int, weights: List[int], values: List[int]) -> int:
    n = len(values)
    prev_row = [0] * (k + 1)

    for i in range(n - 1, -1, -1):
        curr_row = [0] * (k + 1)

        for c in range(1, k + 1):

            if weights[i] <= c:
                curr_row[c] = max(values[i] + prev_row[c - weights[i]], prev_row[c])
            else:
                curr_row[c] = prev_row[c]
        
        prev_row = curr_row
    
    return prev_row[k]